In [96]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)

In [97]:
df_lyrics2 = pd.read_csv('files/lyrics.csv')
df_last_album = pd.read_csv('files/lyrics_new_album.csv', index_col=0)

In [98]:
df_last_album['lyrics'] = df_last_album['lyrics'].apply(lambda x : re.sub(r'\[.*?\]', '', x).strip().replace('\n', '  '))


In [99]:
df_last_album['Album'] = 'The Tortured Poets Department'

In [100]:
df_last_album = df_last_album.reindex(columns = ['song', 'Album', 'lyrics'])

In [101]:
df_last_album.columns = ['Song', 'Album', 'Lyrics']

In [102]:
albumes = ['Red (Taylor’s Version)', 
           'Lover',
           '1989 (Deluxe)',
           'Midnights (3am Edition)',
           'Taylor Swift',
           'Speak Now (Deluxe)',
           'Fearless (Taylor’s Version)',
           'reputation',
           'evermore (deluxe version)',
           'folklore (deluxe version)'      
           ]

df_lyrics_filtrado = df_lyrics2[df_lyrics2['Album'].isin(albumes)]

In [103]:
diccionario = {}

for track in df_lyrics_filtrado['Song']:
    
    df_especific = df_lyrics_filtrado[df_lyrics_filtrado['Song'] == track]
    cadena = "  ".join(df_especific['Lyric'])
    diccionario[track] = cadena

In [104]:
df_lyrics_clean = df_lyrics_filtrado.drop_duplicates(subset = 'Song')

In [105]:
lista_a_dropear = []

for cancion in df_lyrics_clean['Song']:
    
    if '[From the Vault]' in cancion:
        lista_a_dropear.append(cancion)

In [106]:
df_lyrics_clean = df_lyrics_clean[~df_lyrics_clean['Song'].isin(lista_a_dropear)]

In [107]:
df_lyrics_clean['Lyrics'] = df_lyrics_clean.apply(lambda x : diccionario[x['Song']], axis=1)

In [108]:
df_lyrics_clean.drop(columns = ['Lyric', 'Previous Lyric', 'Next Lyric', 'Multiplicity'], inplace=True)

In [109]:
df_lyrics_clean = df_lyrics_clean.reset_index().drop('index', axis=1)

In [110]:
df_lyrics = pd.concat([df_lyrics_clean, df_last_album], ignore_index=True)

In [111]:
words = ['In ', 'On ', 'At ', 'By ', 'For ', 'With ', 'From ', 'To ', 'Into ', 'Onto ', 'Of ', 'About ', 'Above ', 'Below ', 'Under ', 'Over ', 'Through ', 'Between ', 'Among ', 'Within ', 'The ', 'But ', 'A ', 'Oh ']
pattern = r"'[^ ]*"
df_lyrics['Lyrics'] = df_lyrics['Lyrics'].apply(lambda x : re.sub(pattern, '', x))

for word in words:
    df_lyrics['Lyrics'] = df_lyrics['Lyrics'].str.replace(word , ' ')
    df_lyrics['Lyrics'] = df_lyrics['Lyrics'].str.replace(f' {word.lower()}' , ' ')


In [112]:
df_lyrics

,Song,Album,Lyrics
0,22 (Taylor’s Version),Red (Taylor’s Version),It feels like perfect night dress up like hi...
1,All Too Well (Taylor’s Version),Red (Taylor’s Version),"I walked door you, air was cold something i..."
2,Begin Again (Taylor’s Version),Red (Taylor’s Version),Took deep breath mirror He didn like it when ...
3,Come Back...Be Here (Taylor’s Version),Red (Taylor’s Version),"You said it simple way 4 AM, second day How ..."
4,Everything Has Changed (Taylor’s Version),Red (Taylor’s Version),(You good go?) All I knew this morning when I...
...,...,...,...
152,Cassandra,The Tortured Poets Department,I was my new house placing daydreams Patchi...
153,Peter,The Tortured Poets Department,"Forgive me, Peter My lost fearless leader ..."
154,The Bolter,The Tortured Poets Department,"all accounts, she almost drowned When she wa..."
155,Robin,The Tortured Poets Department,"Long may you reign You an animal, you are b..."


In [113]:
df_lyrics.to_csv('files/taylor_full_lyrics.csv', index=False)